In [1]:
import sys
import numpy as np
sys.path.append("../src")

In [2]:
from data_processing import read_BankChurners
from data_processing import remove_outlier
from data_processing import minmax_scaler
from data_processing import feature_encode
from data_processing import encode_label

In [3]:
"""
def read_BankChurners(path):
    dtypes = [
    ('CLIENTNUM', int),
    ('Attrition_Flag', 'U20'),  
    ('Customer_Age', int),
    ('Gender', 'U10'),
    ('Dependent_count', int),
    ('Education_Level', 'U30'),
    ('Marital_Status', 'U20'),
    ('Income_Category', 'U20'),
    ('Card_Category', 'U20'),
    ('Months_on_book', int),
    ('Total_Relationship_Count', int),
    ('Months_Inactive_12_mon', int),
    ('Contacts_Count_12_mon', int),
    ('Credit_Limit', float),
    ('Total_Revolving_Bal', int),
    ('Avg_Open_To_Buy', float),
    ('Total_Amt_Chng_Q4_Q1', float),
    ('Total_Trans_Amt', int),
    ('Total_Trans_Ct', int),
    ('Total_Ct_Chng_Q4_Q1', float),
    ('Avg_Utilization_Ratio', float)
]
    data = np.genfromtxt(
    path, 
    delimiter=',', 
    dtype=dtypes, 
    skip_header=1,   
    encoding='utf-8'
)
    col_names = [dtype[0] for dtype in dtypes]
    return data, col_names
"""
path = '../data/raw/BankChurners.csv'
data, col_names = read_BankChurners(path)


In [4]:
numeric_cols = [name for name in data.dtype.names if data.dtype[name].kind in ('i', 'f')]
str_cols = [name for name in data.dtype.names if data.dtype[name].kind in ('U')]

for col in str_cols:
    data[col] = np.char.strip(data[col], '"')

In [5]:
data

array([(768805383, 'Existing Customer', 45, 'M', 3, 'High School', 'Married', '$60K - $80K', 'Blue', 39, 5, 1, 3, 12691.,  777, 11914., 1.335,  1144, 42, 1.625, 0.061),
       (818770008, 'Existing Customer', 49, 'F', 5, 'Graduate', 'Single', 'Less than $40K', 'Blue', 44, 6, 1, 2,  8256.,  864,  7392., 1.541,  1291, 33, 3.714, 0.105),
       (713982108, 'Existing Customer', 51, 'M', 3, 'Graduate', 'Married', '$80K - $120K', 'Blue', 36, 4, 1, 0,  3418.,    0,  3418., 2.594,  1887, 20, 2.333, 0.   ),
       ...,
       (716506083, 'Attrited Customer', 44, 'F', 1, 'High School', 'Married', 'Less than $40K', 'Blue', 36, 5, 3, 4,  5409.,    0,  5409., 0.819, 10291, 60, 0.818, 0.   ),
       (717406983, 'Attrited Customer', 30, 'M', 2, 'Graduate', 'Unknown', '$40K - $60K', 'Blue', 36, 4, 3, 3,  5281.,    0,  5281., 0.535,  8395, 62, 0.722, 0.   ),
       (714337233, 'Attrited Customer', 43, 'F', 2, 'Graduate', 'Married', 'Less than $40K', 'Silver', 25, 6, 2, 4, 10388., 1961,  8427., 0.703, 1

In [6]:
for col in numeric_cols:
    data[col] = remove_outlier(data[col])
    data[col] = minmax_scaler(data[col])

data_encoded = data.copy()
mappings = {}
categorical_cols = ["Gender", "Education_Level", "Marital_Status", "Income_Category", "Card_Category"]
for col in categorical_cols:
    arr = data_encoded[col]
    encoded, mapping = feature_encode(arr)
    data_encoded[col] = encoded
    mappings[col] = mapping

data_encoded["Attrition_Flag"] = encode_label(data_encoded["Attrition_Flag"])

In [7]:
mappings

{'Gender': {'F': 0, 'M': 1},
 'Education_Level': {'College': 0,
  'Doctorate': 1,
  'Graduate': 2,
  'High School': 3,
  'Post-Graduate': 4,
  'Uneducated': 5,
  'Unknown': 6},
 'Marital_Status': {'Divorced': 0, 'Married': 1, 'Single': 2, 'Unknown': 3},
 'Income_Category': {'$120K +': 0,
  '$40K - $60K': 1,
  '$60K - $80K': 2,
  '$80K - $120K': 3,
  'Less than $40K': 4,
  'Unknown': 5},
 'Card_Category': {'Blue': 0, 'Gold': 1, 'Platinum': 2, 'Silver': 3}}

In [8]:
np.save("../data/processed/BankChurners_processed", data_encoded)